In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional,Activation
from keras.optimizers import SGD
import math
import keras
from sklearn.metrics import mean_squared_error,mean_absolute_error
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# for dirname, _, filenames in os.walk('/kaggle/output/kaggle/working'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import datetime
import time

pd.set_option('display.max_rows', 10000)

d = {'hour': 3600, 'half_hour': 1800, '15_min': 900}
choose_resolution = 'hour'
resolution = d[choose_resolution]

start = '1/01/2020'
start_ts = time.mktime(datetime.datetime.strptime(start, "%d/%m/%Y").timetuple())/resolution # 1800 to have half hours
end =  '1/01/2021'
end_ts = time.mktime(datetime.datetime.strptime(end, "%d/%m/%Y").timetuple())/resolution

hours = list(np.arange(start_ts, end_ts))
print(len(hours))

df = pd.DataFrame({'time':hours, 'hour in year':list(np.arange(len(hours))), 'occupancy': [0]*len(hours), 'revenue (CHF)':[0]*len(hours), })


csv_path = r'/kaggle/input/cookiex/parkrail-sale-app.csv'

parking = pd.read_csv(csv_path, usecols=['start', 'end'], sep= ';')
print(parking.head())

def find_hour(s, e):
    s_str  = datetime.datetime.fromisoformat(s)
    time_st = (time.mktime(s_str.timetuple()) / resolution) - start_ts

    e_str= datetime.datetime.fromisoformat(e)
    time_e = (time.mktime(e_str.timetuple()) / resolution) - start_ts

    return(time_st, time_e)


for index, r in parking.iterrows():
    s, e = find_hour(r['start'], r['end'])
    try:
        if int(e)- int(s) > 0:
            for i in list(np.arange(int(s), int(e))):
                # ind = df[df['hour in year'] == int(i)].index.values.astype(int)[0]

                df.at[int(i), 'occupancy'] += 1

        else:
            # ind = df[df['hour in year'] == int(s)].index.values.astype(int)[0]
            df.at[int(s), 'occupancy'] += 1
    except  Exception:
        pass



df['revenue (CHF)'] = df['occupancy']/4
df.to_csv('timeseries_'+choose_resolution+'.csv', sep=';', index = False)


In [ ]:
thirty_minutes = pd.read_csv('/kaggle/input/timeseries/timeseries_30minutes.csv',sep= ';' )
hour = pd.read_csv('/kaggle/input/timeseries/timeseries_hourly.csv',sep= ';')
fifteen_minutes = pd.read_csv('/kaggle/input/timeseries/timeseries_15minutes.csv',sep= ';')


In [ ]:
thirty_minutes.head(100)


In [ ]:
thirty_minutes.shape

In [ ]:
import time
hour['time_int'] = hour['time'].astype(int)


In [ ]:
import datetime

datetime.datetime.fromtimestamp(438289).strftime('%Y-%m-%d %H:%M:%S')
# hour['date_time'] = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(hour['time_int']))

In [ ]:
# hour.info


In [ ]:
hour.head(100)


In [ ]:
fifteen_minutes.head(100)

In [ ]:
plt.figure(figsize = (15,5))
plt.plot(hour.occupancy)
plt.title('occupancy')
plt.xlabel ('Time(Hour)')
plt.ylabel ('occupancy')
plt.show()

In [ ]:
plt.figure(figsize = (15,5))
plt.plot(fifteen_minutes.occupancy)
plt.title('occupancy')
plt.xlabel ('Time(15mins)')
plt.ylabel ('occupancy')
plt.show()

In [ ]:
plt.figure(figsize = (15,5))
plt.plot(thirty_minutes.occupancy)
plt.title('occupancy')
plt.xlabel ('Time(30mins)')
plt.ylabel ('occupancy')
plt.show()

In [ ]:

scaler = MinMaxScaler() 
close_price =  hour['occupancy'].values.reshape(-1, 1) 
scaled_close = scaler.fit_transform(close_price)
scaled_close = scaled_close[~np.isnan(scaled_close)] 
scaled_close = scaled_close.reshape(-1, 1) 


In [ ]:
SEQ_LEN = 100 
def to_sequences(data, seq_len):
    d = []
    for index in range(len(data) - seq_len):
        d.append(data[index: index + seq_len])
        return np.array(d)
def preprocess(data_raw, seq_len, train_split):
    data = to_sequences(data_raw, seq_len)
    num_train = int(train_split * data.shape[0])
    X_train = data[:num_train, :-1, :]
    y_train = data[:num_train, -1, :]
    X_test = data[num_train:, :-1, :]
    y_test = data[num_train:, -1, :]
    return X_train, y_train, X_test, y_test
"""Walk forward validation: 
Initial SEQ_LEN is defined above, so, walk forward will be shifting one position to the right and create another sequence.
The process is repeated until all possible positions are used."""
X_train, y_train, X_test, y_test = preprocess(scaled_close, SEQ_LEN, train_split = 0.95) 
# 5% of the data saved for testing.print(X_train.shape, X_test.shape)"""Our model will use 1805 sequences representing 99 hours of Bitcoin price changes each for training. We shall be predicting the price for 96 hours in the future"""

In [ ]:
X_train

In [ ]:
DROPOUT = 0.2 
# 20% Dropout is used to control over-fitting during training
WINDOW_SIZE = SEQ_LEN - 1
model = keras.Sequential()# Input layer
model.add(Bidirectional(LSTM(WINDOW_SIZE, return_sequences=True), input_shape=(WINDOW_SIZE, X_train.shape[-1])))
"""Bidirectional RNNs allows to train on the sequence data in forward and backward direction."""
model.add(Dropout(rate=DROPOUT))# 1st Hidden layer
model.add(Bidirectional(LSTM((WINDOW_SIZE * 2), return_sequences = True)))
model.add(Dropout(rate=DROPOUT))# 2nd Hidden layer
model.add(Bidirectional(LSTM(WINDOW_SIZE, return_sequences=False)))# output layer
model.add(Dense(units=1))
model.add(Activation('linear'))
"""Output layer has a single neuron (predicted Bitcoin price). We use Linear activation function which activation is proportional to the input."""
BATCH_SIZE = 64
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(X_train, y_train, epochs=50, batch_size=BATCH_SIZE, shuffle=False, validation_split=0.1)  #try 50
# shuffle not advisable during training of Time Series

In [ ]:
history.history.keys()

In [ ]:
# # history for loss
# plt.figure(figsize = (10,5))
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# # plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
# # prediction on test data
# y_pred = model.predict(X_test) 
# # invert the test to original values
# y_test_inverse = pd.DataFrame(scaler.inverse_transform(y_test)) 
# # assigning datetime
# y_test_inverse.index = hour.index[-len(y_test):] 
# print('Test data:',)
# print(y_test_inverse.tail(3)); print();# invert the prediction to understandable values
# y_pred_inverse = pd.DataFrame(scaler.inverse_transform(y_pred)) 
# # assigning datetime
# y_pred_inverse.index = y_test_inverse.index 
# print('Prediction data:',)
# print(y_pred_inverse.tail(3))

In [ ]:
# print(f'MAE {mean_absolute_error(y_test, y_pred)}')
# print(f'MSE {mean_squared_error(y_test, y_pred)}')
# print(f'RMSE {np.sqrt(mean_squared_error(y_test, y_pred))}')
# # print(f'R2 {r2_score(y_test, y_pred)}')

In [ ]:
# plt.figure(figsize = (15,5))
# plt.plot(y_test_inverse)
# plt.plot(y_pred_inverse)
# plt.title('Actual vs Prediction plot (Price prediction model)')
# plt.ylabel('price')
# plt.xlabel('date')
# plt.legend(['actual', 'prediction'], loc='upper left')
# plt.show()

In [ ]:
close_data = hour['occupancy'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.70
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = hour['time'][:split]
date_test = hour['time'][split:]

print(len(close_train))
print(len(close_test))

In [ ]:
fifteen_minutes.shape

In [ ]:
close_data_30 = fifteen_minutes['occupancy'].values
close_data_30 = close_data_30.reshape((-1,1))

split_percent = 0.70
split = int(split_percent*len(close_data))

close_train_30 = close_data_30[:split]
close_test_30 = close_data_30[split:]

date_train_30 = fifteen_minutes['time'][:split]
date_test_30 = fifteen_minutes['time'][split:]

print(len(close_train_30))
print(len(close_test_30))

In [ ]:
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
look_back = 15

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [ ]:
look_back = 15

train_generator_30 = TimeseriesGenerator(close_train_30, close_train_30, length=look_back, batch_size=20)     
test_generator_30 = TimeseriesGenerator(close_test_30, close_test_30, length=look_back, batch_size=1)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 5
model.fit_generator(train_generator_30, epochs=num_epochs, verbose=1)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 5
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

In [ ]:
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [ ]:
prediction = model.predict_generator(test_generator_30)

close_train = close_train_30.reshape((-1))
close_test = close_test_30.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train_30,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test_30,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test_30,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Predict Occupancy for 15 mins ",
    xaxis = {'title' : "Time"},
    yaxis = {'title' : "Occupancy"}
)
fig = go.Figure(data=[trace1, trace2,trace3], layout=layout)
fig.show()

In [ ]:
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Predict Occupancy for hour ",
    xaxis = {'title' : "Time"},
    yaxis = {'title' : "Occupancy"}
)
fig = go.Figure(data=[trace1, trace2,trace3], layout=layout)
fig.show()

In [ ]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = hour['time'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

trace1 = go.Scatter(
    x = hour['time'].tolist(),
    y = close_data,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)
layout = go.Layout(
    title = "Predict Occupancy",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Occupancy"}
)

fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()


## Recurrent Neural Networks
In a recurrent neural network we store the output activations from one or more of the layers of the network. Often these are hidden later activations. Then, the next time we feed an input example to the network, we include the previously-stored outputs as additional inputs. You can think of the additional inputs as being concatenated to the end of the “normal” inputs to the previous layer. For example, if a hidden layer had 10 regular input nodes and 128 hidden nodes in the layer, then it would actually have 138 total inputs (assuming you are feeding the layer’s outputs into itself à la Elman) rather than into another layer). Of course, the very first time you try to compute the output of the network you’ll need to fill in those extra 128 inputs with 0s or something.

Source: [Quora](https://www.quora.com/What-is-a-simple-explanation-of-a-recurrent-neural-network)
<img src="https://cdn-images-1.medium.com/max/1600/1*NKhwsOYNUT5xU7Pyf6Znhg.png">

Source: [Medium](https://medium.com/ai-journal/lstm-gru-recurrent-neural-networks-81fe2bcdf1f9)

Let me give you the best explanation of Recurrent Neural Networks that I found on internet: https://www.youtube.com/watch?v=UNmqTiOnRfg&t=3s

Now, even though RNNs are quite powerful, they suffer from  **Vanishing gradient problem ** which hinders them from using long term information, like they are good for storing memory 3-4 instances of past iterations but larger number of instances don't provide good results so we don't just use regular RNNs. Instead, we use a better variation of RNNs: **Long Short Term Networks(LSTM).**

### What is Vanishing Gradient problem?
Vanishing gradient problem is a difficulty found in training artificial neural networks with gradient-based learning methods and backpropagation. In such methods, each of the neural network's weights receives an update proportional to the partial derivative of the error function with respect to the current weight in each iteration of training. The problem is that in some cases, the gradient will be vanishingly small, effectively preventing the weight from changing its value. In the worst case, this may completely stop the neural network from further training. As one example of the problem cause, traditional activation functions such as the hyperbolic tangent function have gradients in the range (0, 1), and backpropagation computes gradients by the chain rule. This has the effect of multiplying n of these small numbers to compute gradients of the "front" layers in an n-layer network, meaning that the gradient (error signal) decreases exponentially with n while the front layers train very slowly.



<img src="https://cdn-images-1.medium.com/max/1460/1*FWy4STsp8k0M5Yd8LifG_Q.png">

Source: [Medium](https://medium.com/@anishsingh20/the-vanishing-gradient-problem-48ae7f501257)

## Long Short Term Memory(LSTM)
Long short-term memory (LSTM) units (or blocks) are a building unit for layers of a recurrent neural network (RNN). A RNN composed of LSTM units is often called an LSTM network. A common LSTM unit is composed of a cell, an input gate, an output gate and a forget gate. The cell is responsible for "remembering" values over arbitrary time intervals; hence the word "memory" in LSTM. Each of the three gates can be thought of as a "conventional" artificial neuron, as in a multi-layer (or feedforward) neural network: that is, they compute an activation (using an activation function) of a weighted sum. Intuitively, they can be thought as regulators of the flow of values that goes through the connections of the LSTM; hence the denotation "gate". There are connections between these gates and the cell.

The expression long short-term refers to the fact that LSTM is a model for the short-term memory which can last for a long period of time. An LSTM is well-suited to classify, process and predict time series given time lags of unknown size and duration between important events. LSTMs were developed to deal with the exploding and vanishing gradient problem when training traditional RNNs.



<img src="https://cdn-images-1.medium.com/max/1600/0*LyfY3Mow9eCYlj7o.">

Source: [Medium](https://codeburst.io/generating-text-using-an-lstm-network-no-libraries-2dff88a3968)

Good explanation: https://medium.com/deep-math-machine-learning-ai/chapter-10-1-deepnlp-lstm-long-short-term-memory-networks-with-math-21477f8e4235


## Components of LSTMs
So the LSTM cell contains the following components
* Forget Gate “f” ( a neural network with sigmoid)
* Candidate layer “C"(a NN with Tanh)
* Input Gate “I” ( a NN with sigmoid )
* Output Gate “O”( a NN with sigmoid)
* Hidden state “H” ( a vector )
* Memory state “C” ( a vector)

* Inputs to the LSTM cell at any step are X<sub>t</sub> (current input) , H<sub>t-1</sub> (previous hidden state ) and C<sub>t-1</sub> (previous memory state).  
* Outputs from the LSTM cell are H<sub>t</sub> (current hidden state ) and C<sub>t</sub> (current memory state)